<a href="https://colab.research.google.com/github/Alan-G-S-Oliveira/Trabalho-de-logica/blob/main/Puzzle/Solver_%C3%81rvores_e_Barracas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Puzzle: Árvores e barracas

- Algumas referências do problema:
  1. [https://www.puzzle-tents.com]
  2. ![](https://mathequalslove.net/wp-content/uploads/2017/04/Tents-and-Trees-Puzzles-1.png.webp)

- Instâncias
  1. ![](https://i.imgur.com/txzPcBe.png)
  2. ![](https://mathequalslove.net/wp-content/uploads/2017/04/Tents-and-Trees-Puzzles-2.png.webp)

In [ ]:
!pip install z3-solver

import functools
import itertools
from z3 import *

s=Solver()
# Construção da instância
n = 8
tabuleiro =[
    [0,0,1,0,0,1,0,0],
    [0,0,0,0,1,1,0,1],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [1,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,1],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,1,0],
]

bpl = [2,1,2,0,4,0,0,3]
bpc = [0,1,2,1,2,2,1,3]

    # Modelar as restrições

#cria as formulas atomicas e as auxiliares
b = [ [Bool('b' + str(i) + str(j)) for j in range(n)] for i in range(n)]
au=[]
au2=[]

#cria as formulas aparte do fato de ter ou não uma arvore na posição
for i in range(n):
  for j in range(n):      #no caso de haver uma arvore na posição
    if tabuleiro[i][j] == 1:
        s.add(Not(b[i][j]))
        if i!=0:
            au.append(b[i-1][j])
        if i!=n-1:
            au.append(b[i+1][j])
        if j!=0:
            au.append(b[i][j-1])
        if j!=n-1:
            au.append(b[i][j+1])
        s.add(Or(au))
        au.clear()
    else:                  #no caso de ser um espeço
        if i!=0:
            s.add(Not(And(b[i][j],b[i-1][j])))
            au.append(tabuleiro[i-1][j]==1)
        if i!=n-1:
            s.add(Not(And(b[i][j],b[i+1][j])))
            au.append(tabuleiro[i+1][j]==1)
        if j!=0:
            s.add(Not(And(b[i][j],b[i][j-1])))
            au.append(tabuleiro[i][j-1]==1)
        if j!=n-1:
            s.add(Not(And(b[i][j],b[i][j+1])))
            au.append(tabuleiro[i][j+1]==1)
        if i!=0 and j!=0:
            s.add(Not(And(b[i][j],b[i-1][j-1])))
        if i!=0 and j!=n-1:
            s.add(Not(And(b[i][j],b[i-1][j+1])))
        if i!=n-1 and j!=n-1:
            s.add(Not(And(b[i][j],b[i+1][j+1])))
        if i!=n-1 and j!=0:
            s.add(Not(And(b[i][j],b[i+1][j-1])))
        s.add(Implies(Not(Or(au)),Not(b[i][j])))
        au.clear()

#cria as formulas de acordo com o numero barracas que deve ter em cada linha
for i in range(n):
  for j in range(n):
    au.append(b[i][j])
  if bpl[i] ==0:
    aux = (z3.Or(au))
    s.add(z3.Not(aux))
    au.clear()
    continue
  au2 = list(itertools.combinations(au, bpl[i]))

  lista_2 = []

  for j in range(len(au2)):
    lista_2.append(z3.And(au2[j]))

  lista_3 = (z3.Or(lista_2))

  lista_4 = []

  for j in range(len(lista_2)):
    for k in range(len(lista_2)):
      if j != k:
        lista_4.append(Implies(lista_2[j], Not(lista_2[k])))

  lista_5 = And(lista_4)
  lista_6 = And(lista_5, lista_3)

  s.add(lista_6)
  au.clear()
  au2.clear()
  lista_2.clear()
  lista_4.clear()

#cria as formulas de acordo com o numero barracas que deve ter em cada coluna
for i in range(n):
  for j in range(n):
    au.append(b[j][i])
  if bpc[i] == 0:
    aux = (z3.Or(au))
    s.add(z3.Not(aux))
    au.clear()
    continue
  au2 = list(itertools.combinations(au, bpc[i]))

  for j in range(len(au2)):
    lista_2.append(z3.And(au2[j]))

  lista_3 = (z3.Or(lista_2))

  lista_4 = []

  for j in range(len(lista_2)):
    for k in range(len(lista_2)):
      if j != k:
        lista_4.append(Implies(lista_2[j], Not(lista_2[k])))

  lista_5 = And(lista_4)
  lista_6 = And(lista_5, lista_3)

  s.add(lista_6)
  au.clear()
  au2.clear()
  lista_2.clear()
  lista_4.clear()

#testa se a somatoria das barracas de cada linha e de cada coluna é igual ao numero de arvores

qtd = 0

for i in range(n):
  for j in range(n):
    if tabuleiro[i][j] == 1:
      qtd = qtd + 1

if qtd != sum(bpl) and qtd != sum(bpc):
  s.add(False)

#diz se é satisfazivel
print(s.check())

#imprime o tabuleiro resolvido caso seja satisfazivel
if s.check() == sat:
  modelo=s.model()
  for i in range(n):
      for j in range(n):
         if tabuleiro[i][j] == 1:
             print("A", end=" ")
         elif modelo.evaluate(b[i][j]):
             print("B", end=" ")
         else:
             print("0", end=" ")
      print()

